# Hyperparameter_tunning_Deep_Learning

In [ ]:
import numpy as np
import pandas as pd


In [ ]:
df = pd.read_csv('/content/diabetes.csv')
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


## check the correlation of input features on the output feature.

In [ ]:
df.corr()['Outcome']

,Outcome
Pregnancies,0.221898
Glucose,0.466581
BloodPressure,0.065068
SkinThickness,0.074752
Insulin,0.130548
BMI,0.292695
DiabetesPedigreeFunction,0.173844
Age,0.238356
Outcome,1.000000


In [ ]:
X = df.iloc[:,:-1].values
y = df.iloc[:,-1].values

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [ ]:
X = scaler.fit_transform(X)

In [ ]:
X.shape

(768, 8)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train,y_test = train_test_split(X,y, test_size = 0.2, random_state = 42)

In [ ]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense, Dropout


In [ ]:
model = Sequential()
model.add(Dense(32,activation = 'relu', input_dim = 8))
model.add((Dense(1, activation='sigmoid')))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
model.compile(optimizer='adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
model.fit(X_train, y_train, batch_size=32, epochs = 30, validation_data=(X_test, y_test))

Epoch 1/30
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.8096 - loss: 0.4076 - val_accuracy: 0.7468 - val_loss: 0.5706
Epoch 2/30
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8195 - loss: 0.3851 - val_accuracy: 0.7403 - val_loss: 0.5702
Epoch 3/30
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8180 - loss: 0.3594 - val_accuracy: 0.7403 - val_loss: 0.5695
Epoch 4/30
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8093 - loss: 0.4249 - val_accuracy: 0.7403 - val_loss: 0.5693
Epoch 5/30
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8233 - loss: 0.3750 - val_accuracy: 0.7468 - val_loss: 0.5704
Epoch 6/30
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8513 - loss: 0.3578 - val_accuracy: 0.7403 - val_loss: 0.5661
Epoch 7/30
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8223 - loss: 0.3730 - val_accuracy: 0.7403 - val_loss: 0.5669
Epoch 8/30
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8287 - loss: 0.3907 - val_accuracy: 0.7403 - val_loss

# Start Hyper Parameter Tunning

https://keras.io/keras_tuner/getting_started/

In [ ]:
pip install keras-tuner --upgrade

In [ ]:
import keras_tuner as kt

In [ ]:
def build_model(hp):

  model = Sequential()

  model.add(Dense(32, activation='relu', input_dim=8))
  model.add(Dense(1,activation='sigmoid'))

  optimizer_1 = hp.Choice('optimizer', values = ['adam','sgd','rmsprop','adadelta'])

  model.compile(optimizer= optimizer_1, loss='binary_crossentropy', metrics = ['accuracy'])

  return model




In [ ]:
tuner = kt.RandomSearch(build_model,
                        objective = 'val_accuracy',
                        max_trials = 5)

Reloading Tuner from ./untitled_project/tuner0.json


In [ ]:
tuner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

In [ ]:
tuner.search_space_summary()

Search space summary
Default search space size: 1
optimizer (Choice)
{'default': 'adam', 'conditions': [], 'values': ['adam', 'sgd', 'rmsprop', 'adadelta'], 'ordered': False}


In [ ]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'rmsprop'}

In [ ]:
model = tuner.get_best_models(num_models=1)[0]

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 6 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [ ]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 32)                  │             288 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 321 (1.25 KB)

 Trainable params: 321 (1.25 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
tuner.results_summary()

Results summary
Results in ./untitled_project
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 2 summary
Hyperparameters:
optimizer: rmsprop
Score: 0.7532467246055603

Trial 1 summary
Hyperparameters:
optimizer: adam
Score: 0.7402597665786743

Trial 3 summary
Hyperparameters:
optimizer: sgd
Score: 0.7272727489471436

Trial 0 summary
Hyperparameters:
optimizer: adadelta
Score: 0.5714285969734192


In [ ]:
model.fit(X_train,y_train, batch_size=32, epochs=100, initial_epoch = 6, validation_data=(X_test,y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.7587 - loss: 0.5789 - val_accuracy: 0.7662 - val_loss: 0.5547
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7388 - loss: 0.5572 - val_accuracy: 0.7597 - val_loss: 0.5378
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7722 - loss: 0.5308 - val_accuracy: 0.7662 - val_loss: 0.5274
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7517 - loss: 0.5149 - val_accuracy: 0.7662 - val_loss: 0.5176
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7872 - loss: 0.4762 - val_accuracy: 0.7597 - val_loss: 0.5086
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7702 - loss: 0.4928 - val_accuracy: 0.7662 - val_loss: 0.5029
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7380 - loss: 0.5018 - val_accuracy: 0.7662 - val_loss: 0.5003
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7692 - loss: 0.4812 - val_accuracy: 0.77

## try to find best number of nodes

In [ ]:
def build_model(hp):
  model = Sequential()

  units = hp.Int('units', min_value=8, max_value=128, step = 8)

  model.add(Dense(units = units, activation = 'relu', input_dim =8))
  model.add(Dense(1, activation='sigmoid'))

  model.compile(optimizer = 'rmsprop', loss = 'binary_crossentropy', metrics=['accuracy'])

  return model




In [ ]:
tuner = kt.RandomSearch(build_model,
                        objective = 'val_accuracy',
                        max_trials = 5,
                        directory='my_dir',
                        project_name="helloworld")

In [ ]:
tuner.search(X_train,y_train, epochs = 5, validation_data = (X_test,y_test))

Trial 5 Complete [00h 00m 02s]
val_accuracy: 0.7662337422370911

Best val_accuracy So Far: 0.8116883039474487
Total elapsed time: 00h 00m 14s


In [ ]:
tuner.get_best_hyperparameters()[0].values

{'units': 120}

In [ ]:
model = tuner.get_best_models(num_models=1)[0]

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 6 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [ ]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 120)                 │           1,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │             121 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,201 (4.69 KB)

 Trainable params: 1,201 (4.69 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
tuner.results_summary()

Results summary
Results in my_dir/helloworld
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 0 summary
Hyperparameters:
units: 120
Score: 0.8116883039474487

Trial 4 summary
Hyperparameters:
units: 64
Score: 0.7662337422370911

Trial 3 summary
Hyperparameters:
units: 104
Score: 0.7597402334213257

Trial 1 summary
Hyperparameters:
units: 32
Score: 0.7142857313156128

Trial 2 summary
Hyperparameters:
units: 8
Score: 0.6818181872367859


In [ ]:
model.fit(X_train,y_train, batch_size=32, epochs=100, initial_epoch=6, validation_data=(X_test,y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.8370 - loss: 0.3789 - val_accuracy: 0.7727 - val_loss: 0.5649
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8456 - loss: 0.3758 - val_accuracy: 0.7532 - val_loss: 0.5700
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8463 - loss: 0.3802 - val_accuracy: 0.7532 - val_loss: 0.5692
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8143 - loss: 0.3934 - val_accuracy: 0.7597 - val_loss: 0.5702
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8514 - loss: 0.3663 - val_accuracy: 0.7662 - val_loss: 0.5678
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8289 - loss: 0.3667 - val_accuracy: 0.7727 - val_loss: 0.5700
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8323 - loss: 0.3721 - val_accuracy: 0.7662 - val_loss: 0.5685
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8547 - loss: 0.3443 - val_accuracy: 0.76

# get best number of layerss

In [ ]:
def build_model(hp):

  model = Sequential()
  model.add(Dense(120, activation = 'relu', input_dim=8))

  for i in range(hp.Int('num_layers',min_value=1, max_value=10)):
    model.add(Dense(72,activation='relu'))

  model.add(Dense(1,activation='sigmoid'))

  model.compile(optimizer='rmsprop', loss = 'binary_crossentropy', metrics=['accuracy'])
  return model





In [ ]:
tuner = kt.RandomSearch(build_model,
                        objective = 'val_accuracy',
                        max_trials = 3,
                        directory = 'mydir',
                        project_name = 'num_layers')

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
tuner.search(X_train,y_train, epochs = 5, validation_data=(X_test,y_test))

Trial 3 Complete [00h 00m 05s]
val_accuracy: 0.7857142686843872

Best val_accuracy So Far: 0.7857142686843872
Total elapsed time: 00h 00m 14s


In [ ]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 6}

In [ ]:
model = tuner.get_best_models(num_models=1)[0]

/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 18 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [ ]:
model.fit(X_train,y_train, batch_size=32, epochs=100, initial_epoch=6, validation_data=(X_test,y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9980 - loss: 0.0193 - val_accuracy: 0.6558 - val_loss: 3.3489
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9778 - loss: 0.1044 - val_accuracy: 0.6688 - val_loss: 2.7855
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9984 - loss: 0.0112 - val_accuracy: 0.7078 - val_loss: 2.3787
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 1.0000 - loss: 0.0014 - val_accuracy: 0.7143 - val_loss: 2.5218
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 1.0000 - loss: 4.0335e-04 - val_accuracy: 0.7208 - val_loss: 2.7152
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 1.0000 - loss: 2.6119e-04 - val_accuracy: 0.7208 - val_loss: 2.9087
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 1.0000 - loss: 1.0088e-04 - val_accuracy: 0.7143 - val_loss: 3.1752
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9718 - loss: 0.0920 - val_acc

In [ ]:
def build_model(hp):

  model = Sequential()
  counter = 0

  for i in range(hp.Int('num_layers', min_value = 1, max_value=10)):

    if counter == 0:

      model.add(Dense(hp.Int('units' + str(i), min_value=8, max_value=128, step=8), activation=hp.Choice('activation' + str(i), values=['relu', 'tanh','sigmoid']), input_dim=8))
      model.add(Dropout(hp.Choice('dropout' + str(i), values = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))

    else:
      model.add(Dense(hp.Int('units' + str(i), min_value=8, max_value=128, step=8), activation=hp.Choice('activation' + str(i), values=['relu', 'tanh','sigmoid'])))
      model.add(Dropout(hp.Choice('dropout' + str(i), values = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
    counter+=1

    model.add(Dense(1,activation='sigmoid'))


  model.compile(optimizer= hp.Choice('optimizer', values=['rmsprop','adam','sgd','adam','adadelta']),
                loss = 'binary_crossentropy',
                metrics=['accuracy'])
  return model

In [ ]:
tuner = kt.RandomSearch(build_model,
                        objective = 'val_accuracy',
                        max_trials = 3,
                        directory = 'mydir',
                        project_name = 'final')

In [ ]:
tuner.search(X_train,y_train, epochs = 5, validation_data=(X_test,y_test))

Trial 3 Complete [00h 00m 11s]
val_accuracy: 0.6428571343421936

Best val_accuracy So Far: 0.6428571343421936
Total elapsed time: 00h 00m 25s


In [ ]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 10,
 'units0': 104,
 'activation0': 'sigmoid',
 'dropout0': 0.7,
 'optimizer': 'sgd',
 'units1': 8,
 'activation1': 'relu',
 'dropout1': 0.1,
 'units2': 8,
 'activation2': 'relu',
 'dropout2': 0.1,
 'units3': 8,
 'activation3': 'relu',
 'dropout3': 0.1,
 'units4': 8,
 'activation4': 'relu',
 'dropout4': 0.1,
 'units5': 8,
 'activation5': 'relu',
 'dropout5': 0.1,
 'units6': 8,
 'activation6': 'relu',
 'dropout6': 0.1,
 'units7': 8,
 'activation7': 'relu',
 'dropout7': 0.1,
 'units8': 8,
 'activation8': 'relu',
 'dropout8': 0.1,
 'units9': 8,
 'activation9': 'relu',
 'dropout9': 0.1}

In [ ]:
model = tuner.get_best_models(num_models=1)[0]

In [ ]:
model.fit(X_train,y_train, batch_size=32, epochs=100, initial_epoch=6, validation_data=(X_test,y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 3s 26ms/step - accuracy: 0.6134 - loss: 0.6850 - val_accuracy: 0.6429 - val_loss: 0.6797
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6933 - loss: 0.6728 - val_accuracy: 0.6429 - val_loss: 0.6749
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6868 - loss: 0.6670 - val_accuracy: 0.6429 - val_loss: 0.6706
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6730 - loss: 0.6620 - val_accuracy: 0.6429 - val_loss: 0.6671
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6500 - loss: 0.6654 - val_accuracy: 0.6429 - val_loss: 0.6644
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6304 - loss: 0.6670 - val_accuracy: 0.6429 - val_loss: 0.6619
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6627 - loss: 0.6571 - val_accuracy: 0.6429 - val_loss: 0.6600
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.6575 - loss: 0.6552 - val_accuracy: 0.6